In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('../data/urbanthreads_q4.csv')

In [4]:
df.head(3)

,Order ID,Order Date,Customer ID,Type,SKU,Category,Product Name,Unit Price,Qty,Total,...,Source,City,State,Warehouse,Del Date,Status,Return?,Reason,Ret Rec,Refund
0,UT-100001,2024-10-01,CUST_10001,New,TOP-CAS-M,Tops,Classic Cotton Tee,1299,2,2598,...,Website,Mumbai,Maharashtra,Mumbai,2024-10-05,Delivered,No,NaN,NaN,Not Applicable
1,UT-100002,2024-10-01,CUST_10002,Returning,DRS-FRM-S,Dresses,Floral Midi Dress,2499,1,2499,...,Mobile App,Delhi,Delhi,Delhi,2024-10-04,Delivered,Yes,Size Issue,2024-10-15,Processed
2,UT-100003,2024-01-10,CUST_10003,new,BOT-JNS-L,Bottoms,Slim Fit Jeans,1899,1,1899,...,Website,Bangalore,Karnataka,Bangalore,2024-10-06,Delivered,No,NaN,NaN,Not Applicable


In [6]:
df.shape

(346, 23)